In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = Path("data/model")
RESULTS_DIR = Path("results")
MODEL_DIR = Path("models")
for path in [RESULTS_DIR, MODEL_DIR]:
    path.mkdir(parents=True, exist_ok=True)

VAL_WEEKS = 6
SEQ_LEN = 30
BATCH_SIZE = 512
EPOCHS = 40
LR = 1e-3
PATIENCE = 7
HIDDEN_UNITS = 256
NUM_BLOCKS = 4
NUM_LAYERS_PER_BLOCK = 4
FORECAST_LENGTH = 1
DROPOUT = 0.1
MODEL_PATH = MODEL_DIR / "nbeats_best.pt"

In [ ]:
train_raw = pd.read_parquet(DATA_DIR / "train_fe.parquet").copy()
test_raw = pd.read_parquet(DATA_DIR / "test_fe.parquet").copy()

print(f"Train raw shape: {train_raw.shape}")
print(f"Test raw shape : {test_raw.shape}")

train_raw.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,IsWeekend,IsMonthStart,IsMonthEnd,CompetitionMonthsActive,Promo2WeeksActive,PromoIntervalActive,Lag_1,Lag_7,Rolling_Mean_7,Rolling_Std_7
0,1,3,5530.0,668,1,0,0,1,c,a,...,0,0,0,52.800000,NaN,0,0.0,NaN,NaN,NaN
1,1,4,4327.0,578,1,0,0,1,c,a,...,0,0,0,52.833333,NaN,0,5530.0,NaN,NaN,NaN
2,1,5,4486.0,619,1,0,0,1,c,a,...,0,0,0,52.866667,NaN,0,4327.0,NaN,NaN,NaN
3,1,6,4997.0,635,1,0,0,1,c,a,...,1,0,0,52.900000,NaN,0,4486.0,NaN,NaN,NaN
4,1,1,7176.0,785,1,1,0,1,c,a,...,0,0,0,52.966667,NaN,0,0.0,NaN,NaN,NaN


In [ ]:
TARGET_COL = "Sales"
ID_COL = "Store"
DATE_PARTS = ["Year", "Month", "Day"]
CAT_COLS = ["StateHoliday", "StoreType", "Assortment"]
LEAK_COLS = {TARGET_COL, "Customers", "LogSales", "Date"}
NA_ZERO_COLS = [
    "CompetitionMonthsActive",
    "Promo2WeeksActive",
    "Lag_1",
    "Lag_7",
    "Rolling_Mean_7",
    "Rolling_Std_7",
]

OPTIONAL_TEST_COLS = [TARGET_COL, "Customers", "LogSales"]
for col in OPTIONAL_TEST_COLS:
    if col not in test_raw.columns:
        test_raw[col] = np.nan

df_train = train_raw.copy()
df_test = test_raw.copy()

for frame in [df_train, df_test]:
    date_frame = frame[DATE_PARTS].rename(columns={"Year": "year", "Month": "month", "Day": "day"})
    frame["Date"] = pd.to_datetime(date_frame)
    frame.sort_values([ID_COL, "Date"], inplace=True)
    frame.reset_index(drop=True, inplace=True)

df_train["dataset"] = "train"
df_test["dataset"] = "test"
combined = pd.concat([df_train, df_test], ignore_index=True)

for col in CAT_COLS:
    if col in combined.columns:
        combined[col] = combined[col].astype(str)

for col in NA_ZERO_COLS:
    if col in combined.columns:
        combined[col] = combined[col].fillna(0)

combined = pd.get_dummies(combined, columns=CAT_COLS, drop_first=True)
combined = combined.sort_values([ID_COL, "Date"]).reset_index(drop=True)

df_train = combined[combined["dataset"] == "train"].drop(columns=["dataset"]).reset_index(drop=True)
df_test = combined[combined["dataset"] == "test"].drop(columns=["dataset"]).reset_index(drop=True)

FEATURE_COLS = [col for col in df_train.columns if col not in LEAK_COLS and col != ID_COL]

print(f"Train shape (post-encoding): {df_train.shape}")
print(f"Test shape  (post-encoding): {df_test.shape}")
print(f"Feature count: {len(FEATURE_COLS)}")

In [ ]:
split_date = df_train["Date"].max() - pd.Timedelta(weeks=VAL_WEEKS)

train_main = df_train[df_train["Date"] < split_date].copy()
val_main = df_train[df_train["Date"] >= split_date].copy()

feature_scaler = StandardScaler()
target_scaler = StandardScaler()

train_main[FEATURE_COLS] = feature_scaler.fit_transform(train_main[FEATURE_COLS])
val_main[FEATURE_COLS] = feature_scaler.transform(val_main[FEATURE_COLS])

test_scaled = df_test.copy()
test_scaled[FEATURE_COLS] = feature_scaler.transform(test_scaled[FEATURE_COLS])

train_main[[TARGET_COL]] = target_scaler.fit_transform(train_main[[TARGET_COL]])
val_main[[TARGET_COL]] = target_scaler.transform(val_main[[TARGET_COL]])

print(f"Split date: {split_date.date()}")
print(f"Train rows: {len(train_main):,} | Val rows: {len(val_main):,}")

In [ ]:
def build_sequences(df: pd.DataFrame, feature_cols: list, target_col: str):
    sequences, targets = [], []
    for _, group in df.groupby(ID_COL):
        group = group.sort_values("Date")
        feature_values = group[feature_cols].to_numpy(dtype=np.float32)
        target_values = group[target_col].to_numpy(dtype=np.float32)
        if len(feature_values) <= SEQ_LEN:
            continue
        for start in range(len(feature_values) - SEQ_LEN):
            end = start + SEQ_LEN
            sequences.append(feature_values[start:end])
            targets.append(target_values[end])
    return np.array(sequences), np.array(targets)

X_train, y_train = build_sequences(train_main, FEATURE_COLS, TARGET_COL)
X_val, y_val = build_sequences(val_main, FEATURE_COLS, TARGET_COL)

print(f"Train sequences: {X_train.shape}")
print(f"Val sequences  : {X_val.shape}")

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]

train_ds = SequenceDataset(X_train, y_train)
val_ds = SequenceDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print(f"Batches -> train: {len(train_loader)}, val: {len(val_loader)}")

In [ ]:
class NBeatsBlock(nn.Module):
    def __init__(self, input_dim: int, theta_dim: int, hidden_units: int, num_layers: int, dropout: float):
        super().__init__()
        layers = []
        last_dim = input_dim
        for _ in range(num_layers):
            layers.append(nn.Linear(last_dim, hidden_units))
            last_dim = hidden_units
        self.fc_layers = nn.ModuleList(layers)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.theta_layer = nn.Linear(hidden_units, theta_dim)
        self.backcast_dim = input_dim
        self.forecast_dim = theta_dim - input_dim

    def forward(self, x):
        out = x
        for layer in self.fc_layers:
            out = self.activation(layer(out))
            out = self.dropout(out)
        theta = self.theta_layer(out)
        backcast = theta[:, : self.backcast_dim]
        forecast = theta[:, self.backcast_dim :]
        return backcast, forecast


class NBeats(nn.Module):
    def __init__(
        self,
        seq_len: int,
        num_features: int,
        forecast_length: int,
        hidden_units: int,
        num_blocks: int,
        num_layers: int,
        dropout: float,
    ):
        super().__init__()
        self.backcast_dim = seq_len * num_features
        self.forecast_dim = forecast_length
        theta_dim = self.backcast_dim + self.forecast_dim
        self.blocks = nn.ModuleList(
            [
                NBeatsBlock(
                    input_dim=self.backcast_dim,
                    theta_dim=theta_dim,
                    hidden_units=hidden_units,
                    num_layers=num_layers,
                    dropout=dropout,
                )
                for _ in range(num_blocks)
            ]
        )

    def forward(self, x):
        batch_size = x.size(0)
        residual = x.reshape(batch_size, -1)
        forecast = torch.zeros((batch_size, self.forecast_dim), device=x.device)
        for block in self.blocks:
            backcast, block_forecast = block(residual)
            residual = residual - backcast
            forecast = forecast + block_forecast
        return forecast.squeeze(-1)


model = NBeats(
    seq_len=SEQ_LEN,
    num_features=len(FEATURE_COLS),
    forecast_length=FORECAST_LENGTH,
    hidden_units=HIDDEN_UNITS,
    num_blocks=NUM_BLOCKS,
    num_layers=NUM_LAYERS_PER_BLOCK,
    dropout=DROPOUT,
).to(DEVICE)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(model)
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2, verbose=True
)


def run_epoch(loader, train_mode: bool):
    epoch_loss, epoch_mae, steps = 0.0, 0.0, 0
    model.train(mode=train_mode)
    context = torch.enable_grad() if train_mode else torch.no_grad()
    with context:
        for batch in loader:
            features, targets = [tensor.to(DEVICE) for tensor in batch]
            if train_mode:
                optimizer.zero_grad()
            preds = model(features)
            loss = criterion(preds, targets)
            mae = torch.mean(torch.abs(preds - targets))
            if train_mode:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                optimizer.step()
            epoch_loss += loss.item()
            epoch_mae += mae.item()
            steps += 1
    return epoch_loss / steps, epoch_mae / steps


history = []
best_loss = float("inf")
patience_counter = 0

for epoch in range(1, EPOCHS + 1):
    train_loss, train_mae = run_epoch(train_loader, train_mode=True)
    val_loss, val_mae = run_epoch(val_loader, train_mode=False)
    scheduler.step(val_loss)

    history.append(
        {"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss, "val_mae": val_mae}
    )
    print(
        f"Epoch {epoch:02d} | Train MSE {train_loss:.4f} | Train MAE {train_mae:.4f} | "
        f"Val MSE {val_loss:.4f} | Val MAE {val_mae:.4f}"
    )

    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), MODEL_PATH)
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

In [ ]:
if MODEL_PATH.exists():
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()


def collect_predictions(loader):
    preds_list, targets_list = [], []
    with torch.no_grad():
        for features, targets in loader:
            features = features.to(DEVICE)
            preds = model(features).cpu().numpy()
            preds_list.append(preds)
            targets_list.append(targets.numpy())
    return np.concatenate(preds_list), np.concatenate(targets_list)


def calculate_mape(y_true, y_pred):
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


def calculate_rmspe(y_true, y_pred):
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.sqrt(np.mean(((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2)) * 100


def calculate_smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, np.abs(y_pred - y_true) / denominator)
    return np.mean(ratio) * 100


def calculate_mase(y_true, y_pred, naive_errors):
    if naive_errors <= 0 or np.isnan(naive_errors):
        return np.nan
    return mean_absolute_error(y_true, y_pred) / naive_errors


val_preds_scaled, val_targets_scaled = collect_predictions(val_loader)
val_preds = target_scaler.inverse_transform(val_preds_scaled.reshape(-1, 1)).ravel()
val_targets = target_scaler.inverse_transform(val_targets_scaled.reshape(-1, 1)).ravel()

mae = mean_absolute_error(val_targets, val_preds)
rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
mape = calculate_mape(val_targets, val_preds)
rmspe = calculate_rmspe(val_targets, val_preds)
smape = calculate_smape(val_targets, val_preds)

historical_train = train_raw[train_raw["Date"] < split_date][TARGET_COL].values
if len(historical_train) > 1:
    naive_error = np.mean(np.abs(np.diff(historical_train)))
else:
    naive_error = np.nan
mase = calculate_mase(val_targets, val_preds, naive_error)

print(f"Validation MAE : {mae:,.2f}")
print(f"Validation RMSE: {rmse:,.2f}")
print(f"Validation MAPE: {mape:,.2f}%")
print(f"Validation RMSPE: {rmspe:,.2f}%")
print(f"Validation sMAPE: {smape:,.2f}%")
print(f"Validation MASE: {mase:,.2f}")

In [ ]:
def build_test_sequences(test_df: pd.DataFrame, history_df: pd.DataFrame, feature_cols: list, seq_len: int):
    sequences, store_ids, forecast_dates = [], [], []
    for _, row in test_df.iterrows():
        store_id = row[ID_COL]
        forecast_date = row["Date"]
        end_date = forecast_date - pd.Timedelta(days=1)

        store_history = history_df[
            (history_df[ID_COL] == store_id) & (history_df["Date"] <= end_date)
        ].sort_values("Date")

        if len(store_history) < seq_len:
            continue

        seq_features = store_history[feature_cols].tail(seq_len).to_numpy(dtype=np.float32)
        sequences.append(seq_features)
        store_ids.append(store_id)
        forecast_dates.append(forecast_date)

    return np.array(sequences), store_ids, forecast_dates


scaled_history = df_train.copy()
scaled_history[FEATURE_COLS] = feature_scaler.transform(df_train[FEATURE_COLS])
scaled_history = scaled_history.sort_values([ID_COL, "Date"]).reset_index(drop=True)

X_test_seq, test_store_ids, test_dates = build_test_sequences(test_scaled, scaled_history, FEATURE_COLS, SEQ_LEN)
print(f"Test sequences built: {X_test_seq.shape}")

if len(X_test_seq) == 0:
    raise RuntimeError("No valid test sequences were created. Consider reducing SEQ_LEN or ensuring sufficient history.")


test_ds = SequenceDataset(X_test_seq)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

model.eval()
test_preds_scaled = []
with torch.no_grad():
    for batch in test_loader:
        preds = model(batch.to(DEVICE)).cpu().numpy()
        test_preds_scaled.append(preds)

flat_test_preds_scaled = np.concatenate(test_preds_scaled)
flat_test_preds = target_scaler.inverse_transform(flat_test_preds_scaled.reshape(-1, 1)).ravel()

submission = pd.DataFrame({
    "Store": test_store_ids,
    "Date": test_dates,
    "PredictedSales": flat_test_preds,
})

if "Id" in test_scaled.columns:
    id_lookup = test_scaled[["Store", "Date", "Id"]].drop_duplicates()
    matched_ids = []
    for store, date in zip(test_store_ids, test_dates):
        match = id_lookup[(id_lookup["Store"] == store) & (id_lookup["Date"] == date)]
        matched_ids.append(match["Id"].iloc[0] if not match.empty else np.nan)
    submission["Id"] = matched_ids
    submission = submission[["Id", "Store", "Date", "PredictedSales"]]

output_path = RESULTS_DIR / "nbeats_predictions.csv"
submission.to_csv(output_path, index=False)
print(f"Saved {len(submission)} predictions to {output_path}")